In [1]:
import csv
from estnltk import Text

In [2]:
from estnltk import Span, Layer
from estnltk.taggers import FlattenTagger
from estnltk.taggers import DiffTagger

In [3]:
from grammarextractor.data_processing.measurement_extraction import MeasurementTokenTagger
from grammarextractor.data_processing.measurement_extraction import MeasurementTagger

In [4]:
measurement_token_tagger = MeasurementTokenTagger(output_layer='measurement_tokens')
measurement_tagger = MeasurementTagger(layer_of_tokens='measurement_tokens')

In [5]:
lines = []
with open("examples.csv", "r") as fin:
    reader = csv.reader(fin)
    for row in reader:
        lines.append(row)

In [6]:
texts_with_gold = []
for line in lines:
    
    text = Text(line[0])
    try:
        start = int(line[1])
        end = int(line[2])
        value = float(line[3])
        layer = Layer(name = 'gold', attributes = ['value'], ambiguous = True)
        layer.add_span(Span(start = start, end = end, legal_attributes = ['value'], value = value))
    
    except ValueError:
        layer = Layer(name = 'gold', attributes = ['value'])
    
    text['gold'] = layer
    texts_with_gold.append(text)

In [7]:
tagged_texts = []
for text in texts_with_gold:
    measurement_token_tagger.tag(text)
    measurement_tagger.tag(text)
    tagged_texts.append(text)

In [8]:
tagged_texts[0].measurements

Layer(name=measurements, spans=SL[AS[Annotation(PSA 19 05 2011 -1,32, {'DATE': '19 05 2011', 'OBJECT': 'PSA', 'REGEX_TYPE': 'PSA', 'UNIT': '', 'VALUE': '1.32', 'name': 'MEASUREMENT'})]])

In [9]:
tagged_texts[0].gold

Layer(name=gold, spans=SL[AS[Annotation(PSA 19 05 2011 -1,32, {'value': 1.32})]])

In [10]:
flatten_tagger = FlattenTagger(input_layer='measurements',
                               output_layer='measurements_flat',
                               output_attributes=['name', 'value'],
                               attribute_mapping=(('REGEX_TYPE', 'name'), 
                                                  ('VALUE','value'), 
                                                  )
                               )

In [11]:
r = flatten_tagger.tag(tagged_texts[0])

In [12]:
r

Text(text='PSA 19 05 2011 -1,32 ja 26 09 2011 PSA oli pt-i sõnadel 0,044ng/ml.')

In [13]:
diff_tagger = DiffTagger(layer_a='gold',
                         layer_b='measurements_flat',
                         output_layer='diff',
                         output_attributes=['value']
                        )

In [14]:
status = {}
diff_tagger(r, status).diff

Layer(name=diff, spans=SL[AS[Annotation(PSA 19 05 2011 -1,32, {'input_layer_name': 'gold', 'span_status': 'modified', 'value': 1.32}), Annotation(PSA 19 05 2011 -1,32, {'input_layer_name': 'measurements_flat', 'span_status': 'modified', 'value': '1.32'})]])

In [15]:
status

{'conflicts': 0,
 'extra_annotations': 1,
 'extra_spans': 0,
 'missing_annotations': 1,
 'missing_spans': 0,
 'modified_spans': 1,
 'overlapped': 0,
 'prolonged': 0,
 'shortened': 0,
 'unchanged_annotations': 0,
 'unchanged_spans': 0}

In [16]:
from estnltk.taggers.standard_taggers.diff_tagger import iterate_modified
from estnltk.taggers.standard_taggers.diff_tagger import iterate_extra

In [18]:
for a in iterate_modified(r.diff, 'span_status'):
    a.display()

text,start,end,span_status,input_layer_name,value
"PSA 19 05 2011 -1,32",0,20,modified,gold,1.32
,0,20,modified,measurements_flat,1.32


In [21]:
r.diff[0][1]

Annotation(PSA 19 05 2011 -1,32, {'input_layer_name': 'measurements_flat', 'span_status': 'modified', 'value': '1.32'})